# Robot Code in Detail

Previously, we opened and started to take a look at the `Robot.java` file for our example project, and also cloned our own robot for the 2022 season using git. Now we'll take a look a closer look at the code, as well as the organization, patterns, and design philosophy that underpins it.

## Conceptual Organization

Building and programming a robot can be a complicated task that involves keeping track of a large number of electronics. If we try to handle everything just using the features provided in our `Robot.java` file, we'd quickly see that it becomes unsustainable. 

Instead, to keep our code at least moderately understandable, we can split our code from one giant `Robot` class to several individual classes that each handle a specific task, in line with the [seperation of concerns](https://en.wikipedia.org/wiki/Separation_of_concerns) principle.

The benefits of this are that:
- if something doesn't work, we can more easily find where the problem came from
- someone reading our code (including us in the future) can better understand what the code does
- it's much nicer to look at :)


Luckily, WPILib provides us with such a design through their `"Command-Based" Programming` methodology

For our robot, we can break down the task of `running the robot` into 4 seperate layers, those being

1. getting user input
2. turning input into desired actions
3. executing actions on relevant parts of the robot 
4. making the robot respond physically

Splitting it up this way, we would also need some additional system that ties them together.

A simple hypothetical example would be: if (1) a driver pushes a joystick forward on an Xbox controller, (2) it gives a command to drive the robot. This command then (3) tells the subsystem controlling the motors to spin them a certain way, which then (4) makes the robot go forwards. 

Going even further, we see that this framework can be used to further break down layers 3 and 2 into individual pieces, called `subsystem`s, and `command`s.

![](https://docs.wpilib.org/en/stable/_images/subsystems-and-commands.drawio.svg)

## Subsystems 

> Subsystems encapsulate lower-level robot hardware (such as motor controllers, sensors, and/or pneumatic actuators), and define the interfaces through which that hardware can be accessed by the rest of the robot code. Subsystems allow users to “hide” the internal complexity of their actual hardware from the rest of their code
>
>\- WPILIB documentation

In essense, subsystems are simple abstraction that group bits of hardware that work to control one core part of the robot (like driving). Then, after creating a subsystem, we can communicate with that subsystem directly instead of managing its individual components.

### Creating a subsystem

as Subsystems are quite general containers, so there's not much that we have to provide. To create a subsystem, we'd create a new class that extends from the `SubsystemBase` class, and optionally provide something to do everytime the command scheduler runs (we'll see this later, but it should be the same freqeuency as `robotPeriodic`)

```java
import edu.wpi.first.wpilibj2.command.SubsystemBase;

public class ExampleSubsystem extends SubsystemBase {
  /** Creates a new ExampleSubsystem. */
  public ExampleSubsystem() {}

  @Override
  public void periodic() {
    // This method will be called once per (~20ms)
  }
}
```

which, if you look at the file in you example project, should be in the `frc\robot\subsystems` folder

### Subsystem Example: Drivetrain
The drivetrain consists of the parts of the robot that move the wheels and drive the robot. 

example: West Coast Drive drivetrain

![](https://i.imgur.com/sCfRoNv.png)

The electronic components for the simple model of the drivetrain above are just those four motors (for the sake of space, the example will just have 1).

A drivetrain subsystem should also have a method that handles the process of moving the motors internally.

So in code, a drivetrain class would look something like: 
```java
import edu.wpi.first.wpilibj2.command.SubsystemBase;
// note this is a 3rd party dependency
import com.ctre.phoenix.motorcontrol.can.TalonFX; 

public class ExDrivetrain extends SubsystemBase {
    // we want to handle the motor controllers only in this 
    //subsystem so they should be private
    private TalonFX motor;
    public ExDrivetrain() {
        motor = new TalonFX(/*motor id*/);
        motor.configFactoryDefault();
        motor.setNeutralMode(NeutralMode.Brake);
    }

    public void drive(boolean isForward){
        // tells the motor controllers to run the motors 
        if (forward) motor.set(1); // full power!!
        else         motor.set(-1); // full reverse!!
    }
}
```

Now that we have a subsystem, we can interact with that subsystem instead of interacting with individual electronic components, making our code a whole lot more understandable.  

However...

Having *just* `Subsystem` as an abstraction still falls short in a big way:

***We still have to manually check each input case and map that to some chain of actions!!***

```java
@Override
public void robotPeriodic() {
    if (/* input case 1*/){
        // do something w/ some subsystems
    } else if (/* input case 2 */) {
        // do something else w/ other subsystems
    } ...
```

We see that with enough cases and complexity, this pattern becomes unsustainable

(note)We also have to manually deal with conflicts in our design; for example if we accidently had 2 input cases that needed to use the same subsystem at the same time.... things would break

## Commands

> Commands are simple state machines that perform high-level robot functions using the methods defined by subsystems. Commands can be either idle, in which they do nothing, or scheduled, in which the scheduler will execute a specific set of the command’s code depending on the state of the command. The CommandScheduler recognizes scheduled commands as being in one of three states: initializing, executing, or ending. Commands specify what is done in each of these states through the initialize(), execute() and end() methods. 
>
> \- WPIlib documentation

Commands are an abstraction over actions/ responsibilities that can be performed at any given time. 

### Creating a Command
To create a command, extend the `CommandBase` class, and override a few important methods, those being `initialize`, `execute`, `end`, and `isFinished`
```java
import edu.wpi.first.wpilibj2.command.CommandBase;

class ExampleCommand extends CommandBase {
  @Override
  public void initialize() {}

  @Override
  public void execute() {}

  @Override
  public void end(boolean interrupted) {}

  @Override
  public boolean isFinished(){ return false; }
}
```

`initialize` : when the command starts executing (similar to the `<x>Init` methods)
  - set up everything needed for the execution of the command

`execute` : while the command is running (similar to the `<x>Periodic` methods)
  - the main thing  the action/command is trying to accomplish

`end` : directly after the command stops running
  - clean up and finalize the command (bring it to a conclusion)
  - the `interrupted` parameter is whether it finished successfully (`false`) or was shut down by the scheduler (`true`)

`isFinished` checks whether the command has accomplished its goal
  - if it returns `true`, then `end` will be called with `false` and execution for the command will stop
  - always returning false makes it never stop unless interrupted

#### Note : Default Behaviors

by default the `end`, `initialize`, and `execute` methods do nothing, and `isFinished` always returns false, so a very simple command that always runs in the background can get away with just overriding `execute`. 

We'll expand on this later by introducing default commands

### Binding Subsystems

A Command like that on its own isn't very helpful, commands need to interact with subsystems to perform some useful action. To specify which subsystems, we need to call the `addRequirements` method of `CommandBase` on all the subsystems that we need (usually in the constructor).

```java
class ExampleCommand extends CommandBase {
    public ExampleCommand(){
        this.addRequirements(subsys1, subsys2, ...);
    }
    @Override
    public void execute() {
        // do stuff
    }
}
```

But how do we get those subsystems??? 

Note : If we created these subsystem in the command, then we'd have duplicate sub-systems for the same components!

We can do it cleanly by passing subsystem objects to the constructor:

```java
class ExampleCommand extends CommandBase {
    public ExampleCommand(SubSystemName subsys1, ...){
        this.addRequirements(subsys1, ...);
    }
}
```

This is an example of a pattern called `dependency injection` where things that the command needs (dependencies) are provided (injected). This helps seperate the creation and usage of out subsystems, in a nod to the principle of `seperation of concerns` from before.

### Default Commands

One particular command usage for a command is to have it always run. These commands often handle core background tasks that are essential to the function of the robot, like a `drive` command that controls robot motion.

for these types of commands, we can make that command a `default` command of the subsystem:

```java
class ExampleCommand extends CommandBase {
    public ExampleCommand(SubSystemName subsys1){
        this.addRequirements(subsys1);
        // this refers to the current object
        subsys1.setDefaultCommand(this);
    }
}
```

If the subsystem is not the requirement of some other command, then this default command will be executed.

### Example Command

```java
public class ClimberCommand extends CommandBase {
    private final Climber climber;
    private final UserAnalog speed;
    // climber is the subsystem
    public ClimberCommand(Climber climber, UserAnalog speed) {
        this.climber = climber;
        this.speed = speed;
        // works b/c addRequirements is a method of CommandBase
        this.addRequirements(climber);
        // making it a default command
        climber.setDefaultCommand(this);
    }

    @Override
    public void execute() {
        climber.extend(speed.get(), false);
    }
}
```

## The Command Scheduler

Now that we have our command, how do we run it? We defined methods for what to do in different cases with `initalize`, `execute`, and `end`, but we never called them anywhere.

Luckily, there is something that handles this, as well as the general logic of commands for us: the Command Scheduler.

Since there should only be 1 instance of the command scheduler, we get it through an `getInstance` static method, e.g.
```java
// this is what is known as a "singleton" object
CommandScheduler.getInstance().run();
```
the CommandScheduler instance has 2 methods of use to us: `schedule` and `run`

`schedule` sets up a command to be run, and checks whether it makes sense to run the command, and then takes appropriate actions. These checks include:
 - whether the command is already scheduled
 - whether the subsystems required aren't already being used
   - if they are being used, whether the other commands be interrupted
  

`run` runs through 1 cycle of the command scheduler, which involves:
 - running each Subsystem `periodic` method
 - executing all scheduled commands
 - seeing which commands have finished and remove them from the schedule

Diagrams for what `schedule` and `run` do respectively (Note 1: `Initalize` is called when it says `Run Command`)(Note 2: `Initialize` isn't called every `run` call for most commands (exception being user input like buttons))

![a](https://docs.wpilib.org/en/stable/_images/scheduler.drawio.svg) ![a](https://docs.wpilib.org/en/stable/_images/scheduler-run-sequence.drawio.svg)

### Using commands

to use our commands, we'd only need to call `schedule` on them when the correct condition is met. For our default commands, this isn't necessary. For example, if we wanted some command to run at the very start, then we could do: 

```java
@Override
public void robotInit() {
    CommandScheduler.getInstance().schedule(
        new SomeCommand(new SomeSubsystem(...))
    );
}
```

We'd also need to reliably run our `CommandScheduler` each cycle, so we would call the `run` method from `robotPeriodic`:

```java
@Override
public void robotPeriodic() {
    CommandScheduler.getInstance().run();
}
```

## Digression : Simplicity & Abstraction 

(cut section maybe or make into notes)

Both by using the command scheduler as opposed to manually managing our subsystems, and by using subsystems instead of working with individual electronics, we saw that our code got a lot cleaner, and we were also able to better understand its logic. 

(Note : It's much easier to understand `drive the robot` as a high level instruction than it is to understand what happens when individual motors are spun)

This idea of code being much more understandable as more and more gets hidden and abstracted away from us is a common one in computing in general. In very many applications, it's often important that we see which parts of the problem are important for us as people to reason with, and which parts can be handled without us needing to think about it explicitly.

## Grouping Commands

We've seen how to work with individual commands, but there are many cases where we want two commands to run at the same time (in parallel), or one after the other (in sequence).

Using the existing features, we can use the `initalize` and `end` methods to schedule parallel and sequential commands respectively, but WPILib offers us a much more elegant solution...

### Sequencing commands

To run one command after another, we can do one of two things.

1. we can a `Command`'s `andThen`/`beforeStarting` methods to run commands after/before some command. (Note that `andThen` can take > 1 commands, but `beforeStarting` only takes 1)

```java
// a chain of commands
command1
  .andThen(command2, command3)
  .beforeStarting(command0);
```


2. we can create a `SequentialCommandGroup` with all the commands we want to run:

```java
new SequentialCommandGroup(
  command0,
  command1,
  command2,
  command3
);
```

the `andThen` and `beforeStarting` methods also return a `SequentialCommandGroup` object, so both these methods can be equally good. `SequentialCommandGroup`s are themselves commands, so they also be used wherever any individual `Command` is needed. (they extend a class which extends CommandBase) 

### Parallelizing Commands

Unlike sequential commands, which just involve running one command, and then running another when the first command finishes, there are many ways to parallelize commands, since those individual commands can end at different times.

#### We Could...

- end when one of the commands ends
- end when all of the commands end
- end when a specific command ends

#### Luckily, we can do all 3

1. to end when the one command finishes, we can use the `raceWith` method
2. to end when the all commands finish, we can use the `alongWith` method
3. to end when a specific command finishes, we can use the `deadlineWith​` method

#### Direct Construction

Similar to sequenced command groups, we can also directly construct parallel command group object instead of using those methods. As there are three ways to parallelize commands, there are 3 different parallel command groups that we can use.

1. ParallelRaceGroup
2. ParallelCommandGroup
3. ParallelDeadlineGroup

The methods from before return values of these types, and values of these parallel groups, like values of sequential command groups, are themselves valid commands

## Inputs

Now that we have made commands that correspond to individual actions and responsibilities, we need some way to know when to schedule commands in response to inputs.

Fortunately, WPILib also provides many ways to get user inputs though outside devices.

For our robots in particular, we only use XBox controllers, so we made our own, even simpler system to work with just those controllers.

![](https://imgs.search.brave.com/LpA8VhuzMVhkD16lVzQtewU__0g1yBfkPw3wiq_dGIA/rs:fit:490:351:1/g:ce/aHR0cHM6Ly93d3cu/dGVjaGllc2hlbHAu/Y29tL3dwLWNvbnRl/bnQvdXBsb2Fkcy8y/MDE0LzEwL3hib3gt/b25lLWNvbnRyb2xs/ZXItZnJlZXplcy5w/bmc)

### Users In An FRC Competition

In an FRC competition, there will be two drivers for each robot. For us, these two drivers are called the `Primary` and `Secondary` drivers/players

### The Controller Class

For our robot, the inputs are handled by the `Controller` class in the `utils` folder.

![](https://i.imgur.com/PLW8afT.png)

To create a binding, we use either the `simpleAxis` method(which represents an input between -1 and 1, like a joystick) or the `simpleButton` method (which represents either on or off, like a button).

We first pass in which controller this refers to, as well as what input to bind.
```java
UserDigital a = Controller.simpleButton(
    Controller.PRIMARY,
    Controller.BUTTONA
);
```
Note that the values are static constants defined as integers, this is an example of an `enumeration`, where one particular value or `state` is represented as an integer. 

The `UserDigital` and `UserAnalog` interfaces are the things that represent those individual bindings. To get the input at a particular time, we call their `get` method (and they should return either a boolean or a double)

```java
// userdigital example
boolean isButtonADown = buttonA.get();
// useranalog example
double leftTriggerOut = leftTrigger.get();
```

### Inputs & Commands

Now that we've created our input bindings, how can we give them to a command, so that they can be properly used? 

Similar to what we did before with subsystems, we can inject them as a constructor parameter!

```java
class ExampleCommand extends CommandBase {
    private SubSystemName sys;
    private UserDigital in;
    public ExampleCommand(SubSystemName subsys1, UserDigital input){
        in = input;
        sys = subsys1;
        this.addRequirements(subsys1);
        // this refers to the current object
        subsys1.setDefaultCommand(this);
    }
    @Override 
    public void execute(){
      if (in.get()) subsys1.doThing();
    }
}
```

## Program Organization

Now that we've seen all the individual parts of our design, we can look at how they fit together (take about why we disregard Main.java)

### Robot.java

(note : remember, us humans aren't aren't that good at thinking of mechanisms as actions ....)

The `Robot` class itself doesn't contain that much in both our Robot codebase as well as in the example project, and this is by design. Because the Robot itself operates off of **time**, and not off of **inputs**, we want to limit the amount of work done in this class. 

All the robot handles are background tasks, such as running the command scheduler (robotPeriodic), and handling phase transitions (teleopInit). The heavy work is done by another class, the `RobotContainer` (which Robot creates in robotInit).

### RobotContainer

the `RobotContainer` class handles the setting up, as well as storage of all the individual subsystems and commands. Generally, subsystems and commands are created in the constructor of the RobotContainer, and it also specifies which commands to run during autonomous.

In addition, the RobotContainer should also creates all of our input bindings, before passing them off to the subsystems and commands

### Constants

Often, we'll have many values that represent specific constants and need to be used throughout our project (example being CAN ids). For those values, we put them in a dedicated Constants class. Since we only need one copy of each, want to access it from everywhere, and don't want to change their values, we'll make them `public static final` fields. 

This can get to be quite long and a bit of an eyesore, so we take advantage of another java feature, and just make them fields within an `interface` instead of a class, which makes them automatically public, static, and final.

### Subfolders

besides the main 3 (4) classes, we want to group all our other classes into meaningful folders (and by extension packages).

Two of these groupings are the `subsystems` and `commands` folders, but we also have in our project a `utils`, or utilities folder containing miscellaneous things that are useful, like our class for managing controllers, and the UserDigital/UserAnalog interfaces

### The in-depth description

https://docs.wpilib.org/en/stable/docs/software/commandbased/structuring-command-based-project.html

### Documentation

- https://first.wpi.edu/wpilib/allwpilib/docs/release/java/index.html
- https://api.ctr-electronics.com/phoenix/release/java/index.html
- https://codedocs.revrobotics.com/java/com/revrobotics/package-summary.html